# Simulating Oscillations

This code simulates damped oscillations using symbolic mathematics with SymPy and numerical solutions with SymPy's lamdify method. It defines the differential equation for a damped oscillator, solves it symbolically, and then evaluates the solution numerically over a specified time range. \
The repo contains 3 files:
- `DampedOscillation.py`: Contains the symbolic definition of the damped oscillator and its solution.
- `Tutorial Learning.ipynb`: A Jupyter notebook that that has my notes when learning the code from an YouTube tutorial. This is very rough, meant only for my personal use.
- `DampedOscillation.ipynb`: This file, meant for additional notes and explanations to the code in `DampedOscillation.py`.

## Structure of the Code
1. First we start by importing the necessary libraries.
2. We define the symbolic variables and functions needed for the damped oscillator.
3. We define the differential equation for the damped oscillator.
4. We solve the differential equation symbolically in the `dsol` function.
5. We create a numerical solution using `sympy.lambdify` function.
6. We plot the results using Matplotlib.

<!-- @format -->

# Physics behind the code
Hide this section if you are not interested in the physics behind the code.


This thing was taught to us in PH1010. We work our way up from a simple harmonic oscillator to a damped harmonic oscillator.

1. First, we face a simple harmonic oscillator, which is described by the equation:
   $$
   \ddot{x} + \omega_n^2 x = 0
   $$
   where $ \ddot{x} $ is the second derivative of $ x $ with respect to time, and $ \omega_n $ is the angular frequency. Usually, $ \omega_n $ is defined as $ \omega_n = \sqrt{\frac{k}{m}} $, where $ k $ is the spring constant, and $ m $ is the mass of the block.
   Solving this equation gives us the solution:
   $$
   x(t) = A \cos(\omega_n t + \phi)
   $$
2. We introduce an external constant force $ F $, gravity for example, acting on the system, which modifies the equation to:
   $$
   \ddot{x} + \omega_n^2 x = F(t)
   $$
   This is basically the same as above, except for a different equilibrium position, and thus different initial conditions(ICS). The solution is not to different from the previous one, except for a shift in the equilibrium position.
3. We then introduce damping, which is a force that opposes the motion of the system, and is proportional to the velocity. This gives us the equation:
   $$
   \ddot{x} + 2 \zeta \omega_n \dot{x} + \omega_n^2 x = 0
   $$
   If the damping force is considered $-bv$ with $b$ as damping coefficient, we have $\zeta = -\frac{b}{2m}$
4. Now, we remove damping, and apply an external force $ F(t) $, which gives us the equation:
   $$
   \ddot{x} + 2 \zeta \omega_n \dot{x} + \omega_n^2 x = F(t)
   $$
   If this $F(t)$ is sinosuidal periodic, with $F(t) = F_d\cos(\omega_d t)$ This is the equation we solve in this code. The solution is:
   $$
   x(t) = A e^{-\zeta \omega_n t} \cos(\omega_d t + \phi)
   $$
   where $ \omega_d = \sqrt{\omega_n^2 - \zeta^2} $ is the damped angular frequency.


# The formula we are solving
The equation we are solving in this code is:
$$\ddot{x} + 2 \zeta \omega_n \dot{x} + \omega_n^2 x = F(t)$$
where:
- $ \ddot{x} $ is the second derivative of $ x $ with respect to time.
- $ \dot{x} $ is the first derivative of $ x $ with respect to time 
- $ \zeta $ is the damping ratio, which is a measure of how much the system is damped. $\zeta = \frac{-b}{2m}$
- $ \omega_n $ is the natural frequency of the system. $\omega_n = \sqrt{\frac{k}{m}}$

In [ ]:
import sympy as smp
import numpy as np
import matplotlib.pyplot as plt

# constants zeta, omega
zeta, omega = smp.symbols("\zeta \omega_n", positive=True, real=True)

# variable t
t = smp.symbols("t", real=True, positive=True)

# Functions x(t), F(t).
x = smp.Function("x")(t)
F = smp.Function("F")(t)


dot_x = smp.diff(x, t)
ddot_x = smp.diff(x, t, 2)


def dsol(ExternalForce):
    '''
    Function to solve our differential equation of damped oscillation
    '''
    D = ddot_x + (2 * zeta * omega * dot_x) + (omega**2) * x
    eqn2 = smp.Eq(D, ExternalForce)
    initconds = {
        x.subs(t, 0): 1,
        dot_x.subs(t, 0): 0,
    }

    # Assume the motion is constrained between 0 and 1(x is instead x/L)
    # And timeperiod of 4 seconds
    return smp.dsolve(eqn2, x, ics=initconds)


def main(zeta_const: int, omega_const: int, t_range: int, Forcem=smp.sin(t)) -> None:
    '''
    The main function to solve our differential equation(using dsolve), and display it on a pyplot graph
    '''
    const = [(zeta, zeta_const), (omega, omega_const)]
    solicsk = dsol(Forcem).subs(const)

    solutionics_num = smp.lambdify(t, solicsk.rhs)
    t_num = np.linspace(0, t_range, 1000)
    solicsk_t = (solutionics_num(t_num))

    # Turns out our dear sympy cant make use of the fact e^in + e^-in = 2 cos n
    # In evaluation, its returning some negligible imaginary part.
    # Since we know our solution needs to be real, we can ignore the imaginary part
    # So that pyplot can plot it properly
    # If you feel that I am scamming you, uncomment this part of the code and check it for yourself

    # print("Original symbolic solution:")
    # print(solicsk.evalf())
    # print(f"Max imaginary part: {np.max(np.abs(np.imag(solicsk_t)))}")

    solicsk_t = np.real(solicsk_t)
    plt.plot(t_num, solicsk_t)

    plt.axhline(0, color='black', linestyle='-', linewidth=1.2)
    plt.axvline(0, color='black', linestyle='-', linewidth=1.2)

    # x limit of amplitude
    plt.plot([0, t_num[-1]], [1, 1], linestyle='--')
    for i in range(int(t_num[-1] // omega_const)):
        # Also, to make lines dotted and fading, use linestyle=':', and alpha for fading.
        plt.plot(
            [omega_const * (i + 1), omega_const * (i + 1)],
            [np.min(solicsk_t), np.max(solicsk_t)],
            linestyle=':',
            color='gray',
            # alpha=1.0 - (i / int(t_num[-1] // 4)),
            linewidth=1.2
        )
    plt.xlabel("t")
    plt.ylabel("x(t)")

    plt.show()

In [ ]:

main(zeta_const=0.1, omega_const=3, t_range=80, Forcem=smp.sin(0.1*t))